In [1]:
# Setting up three graph theory
K4 = ThreeGraphTheory(4, edges=[[0, 1, 2], [1, 2, 3], [0, 2, 3], [0, 1, 3]])
C5 = ThreeGraphTheory.p(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0], [4, 0, 1]])
ThreeGraphTheory.exclude([K4, C5])
TG = ThreeGraphTheory

# Setting up colored theory
TwoColorTheory = combine("2Colors", Color0, Color1, symmetries=NoSymmetry)
TwoColorTheory.exclude([TwoColorTheory(1), TwoColorTheory(1, C0=[0], C1=[0])])
CTG = combine("2ColorThreeGraphs", TG, TwoColorTheory)

# Testing the generation of structures
print("Number of structures without C5, K4")
print("and size 5: ", len(TG.generate(5)), " (should be 20)")
print("and size 6: ", len(TG.generate(6)), " (should be 572)")

print("\nNumber of 2 colored structures without C5, K4")
print("and size 5: ", len(CTG.generate(5)), " (should be 338)")
print("and size 6: ", len(CTG.generate(6)), " (should be 28080)")
TG.printlevel(0)

Number of structures without C5, K4
and size 5:  20  (should be 20)
and size 6:  572  (should be 572)

Number of 2 colored structures without C5, K4
and size 5:  338  (should be 338)
and size 6:  28080  (should be 28080)


In [4]:
a31 = TG.optimize(TG(3, edges=[[0, 1, 2]]), 6, exact=True, denom=2**20, file="certificates/prop_3_1_c5k4")
print("The initial upper bound on the Turan density from Proposition 3.1 is {} ~= {}".format(a31, a31.n()))

The initial upper bound on the Turan density from Proposition 3.1 is 7309337/15728640 ~= 0.464715131123861


In [5]:
# Generate good partition
target_size = 6
all_graphs = TG.generate(target_size, TG(3, ftype=[0, 1, 2], edges=[[0, 1, 2]]))

edge = TG(3, edges=[[0, 1, 2]])
target_typed = 0
for ii, flag in enumerate(all_graphs):
    nfp = flag.not_ftype_points()
    if flag.subflag(points=nfp, ftype_points=[]) != edge:
        continue
    A = 0
    for xx in nfp:
        touch = len(flag.subflag(points=[xx]).blocks("edges")) - 1
        if touch not in [0, 1]:
            A += 1
    if A == 2:
        target_typed += flag
target_flag = target_typed.project()


# Minimum degree
degree = TG(3, ftype=[0], edges=[[0, 1, 2]])
positives = [degree - 0.4641]
# Regularity
degree_difference = TG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]]) - \
TG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]])
positives += [degree_difference, -degree_difference]

# Optimize
ratio = TG.optimize(target_flag, target_size, maximize=False, 
                    positives=positives, exact=True, denom=2**20, 
                    file="certificates/prop_3_2_c5k4"
                   )
a32 = ratio / a31
print("The max-cut ratio returned by Proposition 3.2 is at least {} ~= {}".format(a32, a32.n()))

The max-cut ratio returned by Proposition 3.2 is at least 6017268291/14618674000 ~= 0.411615191022113


In [3]:
target_size = 6

# Edge with correct colors
C = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2])
# Missing edge with good colors
M = CTG(3, edges=[], C0=[0, 1], C1=[2])
# Pointed edge with correct colors, point from color 0
Cp0 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2], ftype=[0])
# Pointed edge with correct colors, point from color 1
Cp1 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2], ftype=[2])


# Edge with bad colors all in color 0
B000 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1, 2], C1=[])
# Edge with bad colors, looking the wrong way
B011 = CTG(3, edges=[[0, 1, 2]], C0=[0], C1=[1, 2])
# Pointed edge with bad colors, point from moving 1 -> 0
Bp0 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1, 2], C1=[], ftype=[0])
# Pointed edge with bad colors, point from moving 0 -> 1
Bp1 = CTG(3, edges=[[0, 1, 2]], C0=[0], C1=[1, 2], ftype=[2])

# Degree equality conditions for various type pairs
degeq_00 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0, 1]) - \
           CTG.p(4, ftype=[1, 0], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0, 1])
degeq_01 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0], C1=[1]) - \
           CTG.p(4, ftype=[0, 1], edges=[[1, 2, 3]], edges_m=[[0, 2, 3]], C0=[0], C1=[1])
degeq_11 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C1=[0, 1]) - \
           CTG.p(4, ftype=[1, 0], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C1=[0, 1])

# Assumptions used in the proof
assums = [
    Cp0 - Bp0, Cp1 - Bp1, # Local optimality
    degeq_00, -degeq_00, degeq_01, -degeq_01, degeq_11, -degeq_11, # Degree regularity
    C - 2/5 # Best fit assumption
]

# Close rational approximation for gamma: 2521/5432
gamma = continued_fraction(2*sqrt(3) - 3).convergent(7)
# Construction
symbolic_constr = CTG.blowup_construction(target_size, 2, edges=[[0, 0, 1]], C0=[[0]], C1=[[1]], symbolic_parts=True).set_sum()
ders = symbolic_constr.derivatives([gamma])

Number of structures without C5, K4
and size 5:  20  (should be 20)
and size 6:  572  (should be 572)

Number of 2 colored structures without C5, K4
and size 5:  338  (should be 338)
and size 6:  28080  (should be 28080)


100%|█████████████████████████████████████████████| 7/7 [00:00<00:00, 10.97it/s]


In [4]:
# First solving the sdp
CTG.solve_sdp(B000 + B011 - (1 - 1/4000)*M, target_size, maximize=True, positives=assums,
              construction=ders, file="certificates/prop_3_3_c5k4_sdp")

Base flags generated, their number is 28080
The relevant ftypes are constructed, their number is 33
Block sizes before symmetric/asymmetric change is applied: [37, 37, 37, 128, 128, 128, 128, 128, 112, 112, 112, 112, 112, 112, 112, 112, 82, 82, 82, 82, 82, 82, 82, 82, 82, 66, 66, 66, 66, 66, 66, 66, 66]


Done with mult table for Ftype on 4 points with edges=(013 023 123), C0=(0 1 2 3), C1=(): : 33it [00:18,  1.83it/s]


Adjusting table with kernels from construction
Tables finished


Done with positivity constraint 8: 100%|██████████| 9/9 [08:22<00:00, 55.88s/it]


Constraints finished
Running SDP. Used block sizes are [26, 10, 36, 26, 10, 21, 106, 39, 88, 56, 72, 40, 88, 21, 106, 32, 80, 68, 44, 32, 80, 68, 44, 68, 44, 32, 80, 68, 44, 32, 80, 36, 46, 50, 32, 54, 28, 82, 36, 46, 35, 46, 54, 28, 50, 32, 36, 46, 22, 44, 22, 44, 42, 24, 42, 24, 42, 24, 42, 24, 21, 44, 22, 44, -28080, -304]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.73e-02 Pobj: -4.6852961e+00 Ad: 1.43e-02 Dobj:  1.0774782e+02 
Iter:  2 Ap: 2.51e-02 Pobj: -1.2052756e+01 Ad: 3.90e-02 Dobj:  1.1238619e+02 
Iter:  3 Ap: 7.98e-02 Pobj: -3.7152273e+01 Ad: 6.91e-02 Dobj:  1.1194943e+02 
Iter:  4 Ap: 1.00e+00 Pobj: -2.4992381e+02 Ad: 2.38e-01 Dobj:  8.5467521e+01 
Iter:  5 Ap: 1.00e+00 Pobj: -2.9064178e+02 Ad: 7.83e-01 Dobj:  1.7369542e+01 
Iter:  6 Ap: 1.00e+00 Pobj: -3.1793513e+02 Ad: 8.93e-01 Dobj:  1.7893695e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -3.4746201e+02 Ad: 9.20e-01 Dobj:  1.5622027e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -4.201170

4.429267091987654e-10

In [37]:
# Second, rounding the sdp solution
CTG.round_solution("certificates/prop_3_3_c5k4_sdp", certificate_file="certificates/prop_3_3_c5k4", 
                   denom=1024*225, slack_threshold=1e-6, kernel_denom=2**20)

Starting the rounding of the result
Flattening X matrices
This took 92.61082577705383s
Correcting flat X matrices
Dimensions:  (601, 72204)
This took 8.055120706558228s
Unflattening X matrices
This took 0.0005853176116943359s
Calculating resulting bound


100%|████████████████████████████████████████| 33/33 [1:31:48<00:00, 166.94s/it]


This took 5511.961145877838s
Final rounded bound is 0


0

In [10]:
(2005756097/10485760000).n()/a31

0.411615191022113

In [8]:
TG.verify("certificates/prop_3_2_c5k4")

The solution is valid, it proves the bound -2005756097/10485760000


-2005756097/10485760000